In [10]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Chrome 드라이버 초기화 및 옵션 설정
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')  # 창 최대화
options.add_argument('--disable-gpu')  # GPU 가속 비활성화
options.add_argument('--no-sandbox')  # 샌드박스 비활성화
options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 비활성화

driver = webdriver.Chrome(options=options)

# CSV 파일 읽기
df = pd.read_csv('nf.csv')

# 결과를 저장할 리스트
price_data = []

# 특정 미용실 검색
search_query = "광진구 갤러리 헤어"

try:
    # 네이버 지도 검색 URL 생성
    encoded_query = search_query.replace(' ', '+')
    naver_maps_url = f"https://map.naver.com/v5/search/{encoded_query}"
    
    # 페이지 로드
    driver.get(naver_maps_url)
    time.sleep(2)  # 대기 시간 감소
    
    # 검색결과 iframe으로 전환
    search_iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "searchIframe"))
    )
    driver.switch_to.frame(search_iframe)
    time.sleep(1)
    
    # 첫 번째 검색 결과 대기 및 클릭
    first_result = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, "#_pcmap_list_scroll_container > ul > li:nth-child(1) > div.qbGlu > div.ouxiq > a > div"))
    )
    driver.execute_script("arguments[0].click();", first_result)
    time.sleep(1)
    
    # 메인 프레임으로 돌아가기
    driver.switch_to.default_content()
    time.sleep(1)
    
    # 상세정보 iframe으로 전환
    entry_iframe = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.ID, "entryIframe"))
    )
    driver.switch_to.frame(entry_iframe)
    time.sleep(1)
    
    # 메뉴 탭 클릭 시도
    try:
        menu_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a.tpj9w:nth-child(2)"))
        )
        driver.execute_script("arguments[0].click();", menu_tab)
        time.sleep(2)
        
        print("메뉴 탭 클릭 완료")
        
    except Exception as e:
        print(f"메뉴 탭 클릭 실패: {str(e)}")
        driver.quit()
        exit()
    
    # 가격 정보 크롤링
    try:
        price_section = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div > div > ul"))
        )
        
        # 전체 텍스트 가져오기
        price_text = price_section.text
        print("수집된 가격 정보:")
        print(price_text)
        
        # 결과 저장
        price_data.append({
            '업소명': "갤러리 헤어",
            '주소': "광진구",
            '가격정보': price_text
        })
        
        # 텍스트 파일로도 저장
        with open('price_info.txt', 'w', encoding='utf-8') as f:
            f.write(price_text)
        
        print("가격 정보 수집 완료")
        
        # 리뷰 탭 클릭 시도
        try:
            review_tab = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "#app-root > div > div > div > div.place_fixed_maintab > div > div > div > div > a:nth-child(3)"))
            )
            driver.execute_script("arguments[0].click();", review_tab)
            time.sleep(2)
            
            print("리뷰 탭 클릭 완료")
            
            # 더보기 버튼 클릭 (반복)
            while True:
                try:
                    # 모든 더보기 버튼 찾기
                    more_buttons = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k1QQ5 > div.NSTUp > div > a > span")
                    
                    if not more_buttons:
                        print("더보기 버튼이 없습니다.")
                        break
                    
                    # 각 더보기 버튼 클릭
                    for button in more_buttons:
                        try:
                            driver.execute_script("arguments[0].click();", button)
                            time.sleep(1)
                            print("더보기 버튼 클릭")
                        except:
                            continue
                    
                    # 페이지 새로고침 후 다시 시도
                    time.sleep(2)
                    
                except Exception as e:
                    print(f"더보기 버튼 클릭 중 오류 발생: {str(e)}")
                    break
            
            # 리뷰 수집
            reviews = []
            review_elements = driver.find_elements(By.CSS_SELECTOR, "#app-root > div > div > div > div:nth-child(6) > div:nth-child(2) > div.place_section.k1QQ5 > div.place_section_content > ul > li")
            
            for review in review_elements:
                try:
                    # 리뷰 링크 클릭
                    review_link = review.find_element(By.CSS_SELECTOR, "div.pui__vn15t2 > a")
                    driver.execute_script("arguments[0].click();", review_link)
                    time.sleep(1)
                    
                    # 리뷰 내용 가져오기
                    review_content = review_link.text
                    

                    # 키워드 가져오기
                    try:
                        keywords = review.find_element(By.CSS_SELECTOR, "div.pui__HLNvmI").text
                        # "--요" 제거 및 쉼표로 구분
                        keywords = keywords.replace('--요', '')
                        keywords = keywords.replace(' ', '')  # 공백 제거
                        keywords = keywords.replace('·', ',')  # 중간점을 쉼표로 변경
                        keywords = keywords.strip(',')  # 앞뒤 쉼표 제거
                    except:
                        keywords = "키워드 없음"
                    
                    reviews.append({
                        '리뷰': review_content,
                        '키워드': keywords
                    })

                    
                    print(f"리뷰: {review_content}")
                    print(f"키워드: {keywords}")
                    print("-" * 50)
                    
                except Exception as e:
                    print(f"리뷰 수집 중 오류 발생: {str(e)}")
                    continue
            
            print(f"총 {len(reviews)}개의 리뷰를 수집했습니다.")
            
            # 리뷰를 CSV 파일로 저장
            reviews_df = pd.DataFrame(reviews)
            reviews_df.to_csv('reviews_with_keywords.csv', index=False, encoding='utf-8-sig')
            
            # 리뷰를 텍스트 파일로도 저장
            with open('reviews.txt', 'w', encoding='utf-8') as f:
                for review in reviews:
                    f.write(f"리뷰: {review['리뷰']}\n")
                    f.write(f"키워드: {review['키워드']}\n")
                    f.write("-" * 50 + "\n")
            
        except Exception as e:
            print(f"리뷰 탭 클릭 실패: {str(e)}")
            driver.quit()
            exit()
            
    except Exception as e:
        print(f"가격 정보 수집 중 오류 발생: {str(e)}")
    
except Exception as e:
    print(f"Error processing {search_query}: {str(e)}")
finally:
    driver.quit()

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(price_data)

# CSV 파일로 저장
result_df.to_csv('price_results.csv', index=False, encoding='utf-8-sig')

메뉴 탭 클릭 완료
수집된 가격 정보:
컷
미용료(커트)
7,000원
일반펌
미용료(파마)
20,000원
가격 정보 수집 완료
리뷰 탭 클릭 완료
더보기 버튼 클릭
더보기 버튼이 없습니다.
리뷰: 가격도 착하고 친절하고 좋아요
키워드: 원하는스타일로잘해줘요친절해요
--------------------------------------------------
리뷰: 친절하고 좋습니다
키워드: 원하는스타일로잘해줘요친절해요
--------------------------------------------------
리뷰: 남성컷트 했는데 친절하고 손이 빠르시고
무난무난 잘 하시는것 같아요 ! 가격도 괜찮고
키워드: 친절해요손이빨라요매장이청결해요가격이합리적이에요
--------------------------------------------------
리뷰: 손빠르게 잘라주십니다
키워드: 시술이꼼꼼해요친절해요손이빨라요
--------------------------------------------------
리뷰: 굿
키워드: 스타일추천을잘해줘요
--------------------------------------------------
리뷰: Good
키워드: 원하는스타일로잘해줘요스타일추천을잘해줘요
--------------------------------------------------
리뷰: 굿굿
키워드: 원하는스타일로잘해줘요스타일추천을잘해줘요트렌디해요고급스러워요손상이적어요
--------------------------------------------------
리뷰: 굿
키워드: 원하는스타일로잘해줘요
--------------------------------------------------
리뷰: 굿
키워드: 원하는스타일로잘해줘요
--------------------------------------------------
리뷰: 청결해요!
키워드: 원하는스타일로잘해줘요
--------------------------------------------------
리뷰: 